In [1]:
import os
import numpy as np
import pandas as pd
import ipywidgets as widgets
import pandas_datareader as pdr
import matplotlib.pyplot as plt

from datetime import datetime
from IPython.display import display, clear_output

In [2]:
os.environ['TIINGO_API_TOKEN'] = ''
plt.style.use('classic')

In [3]:
ticker_dropdown = widgets.Dropdown(
    options=[
        ('AT&T', 'T'),
        ('Apple', 'AAPL'),
        ('Microsoft', 'MSFT'),
        ('Nvidia', 'NVDA'),
        ('Tesla', 'TSLA')
    ],
    description='Company'
)

slow_mavg_slider = widgets.IntSlider(description='Slow Mavg', max=300, value=100)
fast_mavg_slider = widgets.IntSlider(description='Fast Mavg', max=50, value=40)

button = widgets.Button(description='Plot')
output = widgets.Output()

display(
    ticker_dropdown,
    fast_mavg_slider, 
    slow_mavg_slider,
    button
)

def plot(_):
    with output:
        clear_output()
        data = pdr.get_data_tiingo(ticker_dropdown.value, api_key=os.getenv('TIINGO_API_TOKEN')).droplevel(0)
        signals = pd.DataFrame(index=data.index)
        signals['signal'] = 0.0

        signals['Fast Mavg'] = data.close.rolling(window=fast_mavg_slider.value, min_periods=1, center=False).mean()
        signals['Slow Mavg'] = data.close.rolling(window=slow_mavg_slider.value, min_periods=1, center=False).mean()

        signals['signal'][fast_mavg_slider.value:] = np.where(
            signals['Fast Mavg'][fast_mavg_slider.value:] > signals['Slow Mavg'][fast_mavg_slider.value:], 
            1.0, 0.0
        )

        signals['positions'] = signals['signal'].diff()

        fig = plt.figure(figsize=(18, 8), dpi=90)

        ax = fig.add_subplot(111,  ylabel='Price in $')

        data['close'].plot(ax=ax, color='y', lw=2., label=ticker_dropdown.value)

        signals[['Fast Mavg', 'Slow Mavg']].plot(ax=ax, lw=2.)

        ax.plot(signals.loc[signals.positions == 1.0].index, 
                signals['Fast Mavg'][signals.positions == 1.0],
                '^',
                markersize=10, 
                color='g', 
                label='Buy')

        ax.plot(signals.loc[signals.positions == -1.0].index, 
                signals['Fast Mavg'][signals.positions == -1.0],
                'v', 
                markersize=10, 
                color='r', 
                label='Sell')
        
        plt.legend(loc='best')
        plt.title(f'{ticker_dropdown.label} Close Price')
        plt.xlabel('Date')
        plt.grid()
        plt.show()

button.on_click(plot)
widgets.VBox([output])

Dropdown(description='Company', options=(('AT&T', 'T'), ('Apple', 'AAPL'), ('Microsoft', 'MSFT'), ('Nvidia', '…

IntSlider(value=40, description='Fast Mavg', max=50)

IntSlider(value=100, description='Slow Mavg', max=300)

Button(description='Plot', style=ButtonStyle())